In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import string
from time import time
import spacy
from tld import get_tld
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
sns.set_style('whitegrid')
%matplotlib inline

In [2]:
import os 
os.listdir('data')

['alllines.txt', '.DS_Store', 'Shakespeare_data.csv']

In [3]:
lines = list()
with open('data/alllines.txt') as f:
    lines.append(f.readlines())

In [4]:
lines[0][:10]

['"ACT I"\n',
 '"SCENE I. London. The palace."\n',
 '"Enter KING HENRY, LORD JOHN OF LANCASTER, the EARL of WESTMORELAND, SIR WALTER BLUNT, and others"\n',
 '"So shaken as we are, so wan with care,"\n',
 '"Find we a time for frighted peace to pant,"\n',
 '"And breathe short-winded accents of new broils"\n',
 '"To be commenced in strands afar remote."\n',
 '"No more the thirsty entrance of this soil"\n',
 '"Shall daub her lips with her own children\'s blood,"\n',
 '"Nor more shall trenching war channel her fields,"\n']

In [5]:
shakespeare = pd.read_csv('data/Shakespeare_data.csv')
shakespeare.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


# Exploratory Analysis

In [6]:
shakespeare.shape

(111396, 6)

In [7]:
shakespeare['Play'].value_counts()

Hamlet                      4244
Coriolanus                  3992
Cymbeline                   3958
Richard III                 3941
Antony and Cleopatra        3862
King Lear                   3766
Othello                     3762
Troilus and Cressida        3711
A Winters Tale              3489
Henry VIII                  3419
Henry V                     3395
Henry VI Part 2             3334
Romeo and Juliet            3313
Henry IV                    3205
Henry VI Part 3             3138
Alls well that ends well    3083
Measure for measure         2998
Loves Labours Lost          2986
Henry VI Part 1             2983
Richard II                  2937
Merry Wives of Windsor      2831
As you like it              2822
Taming of the Shrew         2806
Merchant of Venice          2802
Julius Caesar               2771
King John                   2766
Titus Andronicus            2726
Much Ado about nothing      2704
Timon of Athens             2662
Twelfth Night               2648
Pericles  

In [8]:
# shakespeare.groupby('Play')['Player'].value_counts()

In [9]:
class TextCleaner(TransformerMixin):
    """Text cleaning to slot into sklearn interface"""

    def __init__(self, remove_stopwords=True, remove_urls=True,
                 remove_puncts=True, lemmatize=True, extra_punct='',
                 custom_stopwords=[], custom_non_stopwords=[],
                 verbose=True, parser='big'):
        """
        INPUT: remove_stopwords - bool - remove is, there, he etc...
               remove_urls - bool - 't www.monkey.com t' --> 't com t'
               remove_punct - bool - all punct and digits gone
               lemmatize - bool - whether to apply lemmtization
               extra_punct - str - other characters to remove
               custom_stopwords - list - add to standard stops
               custom_non_stopwords - list - make sure are kept
               verbose - bool - whether to print progress statements
               parser - str - 'big' or small, one keeps more, and is slower
        OUTPUT: self - **due to other method, not this one
        """
        # Initialize passed Attributes to specify operations
        self.remove_stopwords = remove_stopwords
        self.remove_urls = remove_urls
        self.remove_puncts = remove_puncts
        self.lemmatize = lemmatize

        # Change how operations work
        self.custom_stopwords = custom_stopwords
        self.custom_non_stopwords = custom_non_stopwords
        self.verbose = verbose

        # Set up punctation tranlation table
        self.removals = string.punctuation + string.digits + extra_punct
        self.trans_table = str.maketrans({key: None for key in self.removals})

        #Load nlp model for parsing usage later
        self.parser = spacy.load('en_core_web_sm', 
                                 disable=['parser','ner','textcat'])
        #from spacy.lang.en import English
        if parser == 'small':
            self.parser = spacy.load('en')#English()

        #Add custom stop words to nlp
        for word in self.custom_stopwords:
            self.parser.vocab[word].is_stop = True

        #Set custom nlp words to be kept
        for word in self.custom_non_stopwords:
            self.parser.vocab[word].is_stop = False


    def transform(self, X, y=None):
        """take array of docs to clean array of docs"""
        # Potential replace urls with tld ie www.monkey.com to com
        if self.remove_urls:
            start_time = time()
            if self.verbose:
                print("CHANGING URLS to TLDS...  ", end='')
            X = [self.remove_url(doc) for doc in X]
            if self.verbose:
                print(f"{time() - start_time:.0f} seconds")

        # Potentially remove punctuation
        if self.remove_puncts:
            start_time = time()
            if self.verbose:
                print("REMOVING PUNCTUATION AND DIGITS... ", end='')
            X = [doc.lower().translate(self.trans_table) for doc in X]
            if self.verbose:
                print(f"{time() - start_time:.0f} seconds")

        # Using Spacy to parse text
        start_time = time()
        if self.verbose:
            print("PARSING TEXT WITH SPACY... ", end='')
            
        X = list(self.parser.pipe(X))
        if self.verbose:
            print(f"{time() - start_time:.0f} seconds")

        # Potential stopword removal
        if self.remove_stopwords:
            start_time = time()
            if self.verbose:
                print("REMOVING STOP WORDS FROM DOCUMENTS... ", end='')
            X = [[word for word in doc if not word.is_stop] for doc in X]
            if self.verbose:
                print(f"{time() - start_time:.0f} seconds")


        # Potential Lemmatization
        if self.lemmatize:
            start_time = time()
            if self.verbose:
                print("LEMMATIZING WORDS... ", end='')
            X = [[word.lemma_ for word in doc] for doc in X]
            if self.verbose:
                print(f"{time() - start_time:.0f} seconds")

        # Put back to normal if no lemmatizing happened
        if not self.lemmatize:
            X = [[str(word).lower() for word in doc] for doc in X]

        # Join Back up
        return [' '.join(lst) for lst in X]


    def fit(self, X, y=None):
        """interface conforming, and allows use of fit_transform"""
        return self


    @staticmethod
    def remove_url(text):
        """
        DESCR: given a url string find urls and replace with top level domain
               a bit lazy in that if there are multiple all are replaced by first
        INPUT: text - str - 'this is www.monky.com in text'
        OUTPIT: str - 'this is <com> in text'
        """
        # Define string to match urls
        url_re = '((?:www|https?)(://)?[^\s]+)'

        # Find potential things to replace
        matches = re.findall(url_re, text)
        if matches == []:
            return text

        # Get tld of first match
        match = matches[0][0]
        try:
            tld = get_tld(match, fail_silently=True, fix_protocol=True)
        except ValueError:
            tld = None

        # failures return none so change to empty
        if tld is None:
            tld = ""

        # make this obvsiouyly an odd tag
        tld = f"<{tld}>"

        # Make replacements and return
        return re.sub(url_re, tld, text)

In [10]:
custom_stopwords = []# ['act', 'i', 'ii', 'iii', 'iv', 'v']
shakespeare['PlayerLineClean'] = TextCleaner(
    custom_stopwords=custom_stopwords, remove_urls=False).transform(shakespeare['PlayerLine'])

REMOVING PUNCTUATION AND DIGITS... 0 seconds
PARSING TEXT WITH SPACY... 25 seconds
REMOVING STOP WORDS FROM DOCUMENTS... 1 seconds
LEMMATIZING WORDS... 1 seconds


In [11]:
shakespeare.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine,PlayerLineClean
0,1,Henry IV,NaN,NaN,NaN,ACT I,act
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.,scene london palace
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ...",enter king henry lord john lancaster earl west...
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,",shake wan care
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,",find time frighted peace pant


# Test `DBSCAN` with Word Embeddings

- Issues [described in Stack Overflow](https://stackoverflow.com/questions/25217065/scikit-learn-clustering-text-documents-using-dbscan) that input data can't be sparse with `sklearn`'s implementation of `DBSCAN`.  

In [13]:
# word embeddings
import gensim

sentences = []
for line in shakespeare.PlayerLineClean.tolist():
    sentences.append(line.split())

model = gensim.models.Word2Vec(iter=1, min_count=3, size=500, workers=4)
model.build_vocab(sentences)
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs*2)

(650499, 737418)

In [14]:
word_vectors = model.wv
word_vectors.vectors

array([[ 5.4294765e-03, -1.1735098e-01, -3.9045740e-02, ...,
        -1.5259583e-01,  1.5506427e-01,  1.6219090e-01],
       [-1.4812351e-02, -9.1136366e-02, -3.2857228e-02, ...,
        -1.5083608e-01,  1.6290131e-01,  1.5122457e-01],
       [-2.2959655e-02, -7.5742617e-02, -2.6912160e-02, ...,
        -1.3371272e-01,  1.4572138e-01,  1.3794890e-01],
       ...,
       [ 6.6257309e-04, -1.2850353e-03, -5.0330756e-05, ...,
        -1.8303104e-03,  2.4711546e-03,  1.3462185e-03],
       [-8.8778709e-04, -2.0188512e-03,  2.0569604e-04, ...,
        -3.8304948e-03,  2.7244294e-03,  2.6553897e-03],
       [ 3.7438728e-04, -1.4197803e-03, -7.8081689e-04, ...,
        -9.3241694e-04,  8.0814399e-04,  4.0042153e-04]], dtype=float32)

In [15]:
wv_df = pd.DataFrame(word_vectors.vectors, index=word_vectors.vocab.keys()).T
wv_df.head()

,act,scene,london,palace,enter,king,henry,lord,john,lancaster,earl,westmoreland,sir,walter,blunt,shake,wan,care,find,time,peace,pant,breathe,accent,new,broil,commence,strand,afar,remote,thirsty,entrance,soil,shall,daub,lip,children,blood,trench,war,channel,field,bruise,armed,hoof,hostile,pace,oppose,eye,like,meteor,troubled,heaven,nature,substance,breed,lately,meet,shock,furious,close,civil,butchery,mutual,rank,march,way,opposed,acquaintance,kindr,ally,edge,knife,cut,master,friend,far,sepulchre,christ,soldier,bless,cross,engage,fight,forthwith,power,english,levy,arm,mould,mother,womb,chase,pagan,holy,acre,walkd,foot,fourteen,year,ago,advantage,bitter,purpose,month,old,bootless,tis,tell,let,hear,gentle,cousin,yesternight,council,decree,forward,dear,expedience,liege,haste,hot,question,limit,charge,set,athwart,come,post,wale,loaden,heavy,news,bad,noble,mortimer,lead,man,irregular,wild,glendower,rude,hand,welshman,take,thousand,people,butcher,dead,corpse,misuse,beastly,shameless,transformation,shame,speak,tiding,brake,business,land,matchd,gracious,uneven,unwelcome,north,import,day,gallant,hotspur,young,harry,percy,brave,approve,scot,holmedon,spend,sad,bloody,hour,discharge,artillery,shape,likelihood,bring,heat,pride,contention,horse,uncertain,issue,true,industrious,light,staind,variation,betwixt,seat,hath,smooth,welcome,dougla,discomfited,bold,knight,plain,prisoner,mordake,fife,eld,son,beat,angus,menteith,honourable,spoil,prize,ha,faith,conquest,prince,boast,yea,thou,mak,sin,envy,northumberland,father,blest,theme,honour,tongue,grove,straight,plant,sweet,fortune,minion,whilst,look,praise,riot,dishonour,stain,brow,o,prove,fairy,exchange,child,lie,calld,plantagenet,thought,think,coz,percys,adventure,surprised,use,keep,send,word,uncle,teach,worcester,aspect,make,prune,bristle,crest,youth,dignity,answer,because,awhile,neglect,jerusalem,wednesday,hold,windsor,inform,speed,say,anger,utter,exeunt,ii,apartment,falstaff,hal,lad,art,drinking,sack,thee,supper,sleep,bench,noon,hast,forget,demand,truly,wouldst,know,devil,cup,minute,capon,clock,bawd,dial,sign,blessed,sun,fair,wench,taffeta,reason,shouldst,superfluous,near,purse,moon,seven,star,phoebus,wander,prithee,wag,god,save,thy,grace,wilt,troth,serve,prologue,egg,butter,roundly,marry,squire,night,body,call,thief,beauty,diana,forester,gentleman,shade,good,government,govern,sea,chaste,mistress,countenance,steal,sayest,doth,ebb,flow,proof,gold,resolutely,snatch,monday,dissolutely,tuesday,morning,get,swear,cry,low,ladder,high,ridge,gallow,hostess,tavern,honey,castle,buff,jerkin,robe,durance,mad,quip,plague,pox,reckon,oft,pay,ill,coin,stretch,credit,apparent,heir,stand,england,resolution,fob,rusty,curb,antic,law,hang,shalt,rare,judge,false,mean,hanging,hangman,sort,jump,humour,wait,court,obtain,suit,whereof,lean,wardrobe,sblood,melancholy,cat,lug,bear,lion,lover,lute,drone,bagpipe,hare,unsavoury,simile,comparative,trouble,vanity,commodity,name,buy,rat,street,mark,talk,wisely,regard,didst,wisdom,damnable,iteration,able,corrupt,saint,harm,forgive,little,wicked,life,villain,damn,christendom,tomorrow,jack,zounds,baffle,amendment,pray,vocation,labour,poin,gadshill,match,merit,hole,hell,omnipotent,morrow,ned,monsieur,remorse,sugar,agree,soul,cold,leg,bargain,proverb,cozen,oclock,early,pilgrim,...,sophy,launcelot,gobbo,jews,gramercy,jessica,torchbearer,r,wildcat,huggd,inscription,chooseth,arragon,distinct,ivory,genoa,tripoli,antonios,grossness,salerio,magnifico,balthasar,padua,bellario,envys,waver,daniel,exposition,award,desir,recant,thisbe,carthage,unthrift,stephano,moonlight,beset,evan,got,goot,fery,abraham,conycatch,tam,decrease,bullyrook,singer,pandarus,rogu,phrygian,rugby,fe,dere,vill,brew,gar,fenton,thinkest,nan,actaeon,pottle,recourse,sip,attraction,nayword,largely,excellency,aggravate,curer,frogmore,knog,urinal,fragrant,carryt,buckbasket,farthingale,tattling,welladay,dissolute,throwing,abird,slide,alacrity,pill,polecat,brentford,dishonesty,scrape,herne,urchin,eton,privacy,deanery,budget,trib,eu

In [57]:
wv_df.shape

(500, 8980)

(7508842, 43838620)

In [ ]:
lines_clean